# Extraction code
## Short version of documentation (if you just need to use it):
* Download all the text of stories and store in **one** folder (get all the html out to the outer folder if there's multiple folders nested).
* Replace any "2020_11_20_02_46_09EZnVK6" to the folder that contains the html files of stories (there's two places), replace any "Final_Story_Selection_Spreadsheet.csv" to the csv file that contains the stories need to be processed and their corresponding placenames.
* The result is a file called "output.xlsx", which is the original table with an extra column of extract.

## Longer version of documentation (if you trying to modify the code or understand how it's working):
Import libaray

In [1]:
import os

First we download all the html files from https://cdhrdatasys.anu.edu.au/anfd/web/index.php?r=trove/search, each of the html is a story. The downloaded file may contain multiple folders, we get all the files out the folders first, then we use the below code to rename each files to be their trove ID. dir_ shoule be replaced by the name of the folder that contains all the html files.

In [ ]:
dir_="2020_11_20_02_46_09EZnVK6"
for count, filename in enumerate(os.listdir(dir_)): 
        dst =filename.split('_')[-1]
        print(dst)
        src =dir_+'/'+ filename 
        dst =dir_+'/'+ dst 
          
        # rename() function will 
        # rename all the files 
        os.rename(src, dst) 

We then import another library to extract only texts from the html files, so that no tag or html code will be included in the final extraction.

In [144]:
from bs4 import BeautifulSoup

Implementing the extract rules.

In [174]:
def get_list(listt):
    result_list=[None]*(len(listt))
    for idx, x in enumerate(listt):
        list_new=x.split('.')
        count_temp=list_new[0].split(' ')
        if len(count_temp)>200:
#             list_new[0]=count_temp[-200:].join(' ')
            list_new[0]=''.join(count_temp[-200:])
        count_temp=list_new[-1].split(' ')
        if len(count_temp)>200:
#             list_new[-1]=count_temp[-200:].join(' ')
            list_new[-1]=' '.join(count_temp[-200:])
        if idx==0:
            #if last sentence ends with quotation mark
            if len(list_new[-1])>0 and list_new[-1][0]=='"':
                result_list[idx]=list_new[-1][1:]+placename
#                 print(result_list[idx])
            else:
                result_list[idx]=list_new[-1]+placename
        elif idx==len(listt):
            result_list[idx-1]+=list_new[0]
            result_list[idx-1]+='.'
            if len(list_new)>1:
                #check if the sentence ends with quotation mark
                if len(list_new[1])>0 and list_new[1][0]=='"':
                    result_list[idx-1]='"'+result_list[idx-1]+'"'
        else:
            result_list[idx-1]+=list_new[0]
            result_list[idx-1]+='.'
            if len(list_new)>1:
                #check if the sentence ends with quotation mark
                if len(list_new[1])>0 and list_new[1][0]=='"':
                    result_list[idx-1]='"'+result_list[idx-1]+'"'
            #if last sentence ends with quotation mark
            if len(list_new[-1])>0 and list_new[-1][0]=='"':
                result_list[idx]=list_new[-1][1:]+placename
            else:
                result_list[idx]=list_new[-1]+placename


    #     list_new[0]
    for idx, x in enumerate(result_list):
        result_list[idx]=x.strip()
        result_list[idx]='... '+result_list[idx]+' ...'
#         print(result_list[idx])
    return result_list

Here we started to do extraction for our spreadsheet, replace 'Final_Story_Selection_Spreadsheet.csv' to the name of the spreadsheet.

In [100]:
import pandas as pd

In [164]:
source=pd.read_csv('Final_Story_Selection_Spreadsheet.csv',encoding='latin-1')
source_use=source[['Index','Unique_id','Placename','Character_Location']]
table=source_use.sort_values(by=['Unique_id', 'Placename','Character_Location'])
table['extraction']=None
table

,Index,Unique_id,Placename,Character_Location,extraction
657,658,18328955,Bohemia,2383,None
659,660,18328955,Cuzco,2711,None
652,653,18328955,Melbourne,715,None
655,656,18328955,Melbourne Cafe,1162,None
654,655,18328955,New Guinea,1028,None
...,...,...,...,...,...
451,452,189261419,Netherby,18771,None
452,453,189261419,Netherby,19144,None
453,454,189261419,Netherby,24289,None
447,448,189261419,Queenscliffe,8959,None


Replace "2020_11_20_02_46_09EZnVK6/" to the name of the containing folder (of all the processed html files). Below code go through every row of the table and find each of them their own extract.

In [ ]:
file=""
dic={}
for index, row in table.iterrows():
    print(row['Unique_id'], row['Placename'])
    temp_file="2020_11_20_02_46_09EZnVK6/"+str(row['Unique_id'])+".html"
    if temp_file!=file:
        file=temp_file
        f=open(file,"rb")
        txt=f.read()
        soup = BeautifulSoup(txt)
        t=soup.get_text()
        placename=str(row['Placename'])
        listt=t.split(placename)
        dic={}
        dic[placename]=get_list(listt)
        row['extraction']=dic[placename][0]
        table.loc[index,'extraction']=dic[placename][0]
        dic[row['Placename']]=dic[row['Placename']][1:]
    else:
        
        if row['Placename'] in dic.keys():
#             print(dic.keys())
#             print(dic[row['Placename']])
#             row['extraction']=dic[placename][0]
            table.loc[index,'extraction']=dic[row['Placename']][0]
            dic[row['Placename']]=dic[row['Placename']][1:]
        else:
            placename=str(row['Placename'])
            listt=t.split(placename)
            dic[placename]=get_list(listt)
            row['extraction']=dic[placename][0]
            table.loc[index,'extraction']=dic[placename][0]
            dic[row['Placename']]=dic[row['Placename']][1:]
#     print(row['Unique_id'], row['Placename'])

Have a look at the result, make sure everything looks perfect.

In [178]:
result=table.sort_values(by=['Index'])
result

,Index,Unique_id,Placename,Character_Location,extraction
0,1,66216131,England,486,... turn to England immediately to fulfil cert...
1,2,66216131,England,1556,"... ""Are you not my only love? Have I ever giv..."
2,3,66216131,Rome,1931,"... So Katie tried to feel content, and when t..."
3,4,66216131,Gringigal,2427,"... The winter was spent at Gringigal gone, a ..."
4,5,66216131,Philip,2601,"... ""You know I am not -beautiful like some gi..."
...,...,...,...,...,...
838,839,36060104,moon,846,... An hour or so had been supping by at tibia...
839,840,36060104,England,4823,... - who wrote of the death of her sister — t...
840,841,36060104,England,5603,"... Not poppy, nor mandragoia, nor all the dro..."
841,842,36060105,Jericho,3097,... My preacher had selected the subject of bi...


Append the result to the last column of the original table.

In [180]:
source['Extraction']=result['extraction']
source

,Index,Trove_ID,date-date-date,Chapter,Chapter_Title,Placename,Character_Location,Unique_id,Common Title,Publication Title,...,Nationality,Nationality Details,Author Details,Inscribed Gender,Inscribed Nationality,Signature,Name Category,Unnamed: 35,Unnamed: 36,Extraction
0,1,7418,28-4-1894,KATIE BRYCE'S TWO CHRISTMAS DAYS. I.,KATIE BRYCE'S TWO CHRISTMAS DAYS. I.,England,486,66216131,Katie Bryce's Two Christmas Days,Katie Bryce's Two Christmas Days,...,Australian,NaN,NaN,Female,NaN,NaN,"Honorific, initials and surname",NaN,NaN,... turn to England immediately to fulfil cert...
1,2,7418,28-4-1894,KATIE BRYCE'S TWO CHRISTMAS DAYS. I.,KATIE BRYCE'S TWO CHRISTMAS DAYS. I.,England,1556,66216131,Katie Bryce's Two Christmas Days,Katie Bryce's Two Christmas Days,...,Australian,NaN,NaN,Female,NaN,NaN,"Honorific, initials and surname",NaN,NaN,"... ""Are you not my only love? Have I ever giv..."
2,3,7418,28-4-1894,KATIE BRYCE'S TWO CHRISTMAS DAYS. I.,KATIE BRYCE'S TWO CHRISTMAS DAYS. I.,Rome,1931,66216131,Katie Bryce's Two Christmas Days,Katie Bryce's Two Christmas Days,...,Australian,NaN,NaN,Female,NaN,NaN,"Honorific, initials and surname",NaN,NaN,"... So Katie tried to feel content, and when t..."
3,4,7418,28-4-1894,KATIE BRYCE'S TWO CHRISTMAS DAYS. I.,KATIE BRYCE'S TWO CHRISTMAS DAYS. I.,Gringigal,2427,66216131,Katie Bryce's Two Christmas Days,Katie Bryce's Two Christmas Days,...,Australian,NaN,NaN,Female,NaN,NaN,"Honorific, initials and surname",NaN,NaN,"... The winter was spent at Gringigal gone, a ..."
4,5,7418,28-4-1894,KATIE BRYCE'S TWO CHRISTMAS DAYS. I.,KATIE BRYCE'S TWO CHRISTMAS DAYS. I.,Philip,2601,66216131,Katie Bryce's Two Christmas Days,Katie Bryce's Two Christmas Days,...,Australian,NaN,NaN,Female,NaN,NaN,"Honorific, initials and surname",NaN,NaN,"... ""You know I am not -beautiful like some gi..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
838,839,5421,2-12-1881,VI,A QUEER CHRISTMAS-NIGHT'S VISITOR.,moon,846,36060104,A Queer Christmas Adventure,A Queer Christmas Adventure,...,Australian,NaN,ADB,Unknown,Australian,"""The Australian Abroad""",Attributed with signature,NaN,NaN,... An hour or so had been supping by at tibia...
839,840,5421,2-12-1881,VI,A QUEER CHRISTMAS-NIGHT'S VISITOR.,England,4823,36060104,A Queer Christmas Adventure,A Queer Christmas Adventure,...,Australian,NaN,ADB,Unknown,Australian,"""The Australian Abroad""",Attributed with signature,NaN,NaN,... - who wrote of the death of her sister — t...
840,841,5421,2-12-1881,VI,A QUEER CHRISTMAS-NIGHT'S VISITOR.,England,5603,36060104,A Queer Christmas Adventure,A Queer Christmas Adventure,...,Australian,NaN,ADB,Unknown,Australian,"""The Australian Abroad""",Attributed with signature,NaN,NaN,"... Not poppy, nor mandragoia, nor all the dro..."
841,842,5421,2-12-1881,V,A CHRISTMAS DAY'S STROLL.,Jericho,3097,36060105,A Queer Christmas Adventure,A Queer Christmas Adventure,...,Australian,NaN,ADB,Unknown,Australian,"""The Australian Abroad""",Attributed with signature,NaN,NaN,... My preacher had selected the subject of bi...


Finally we output the table to an excel table with the name "output.xlsx", you can change it to anything you like if you want to.

In [183]:
source.to_excel("output.xlsx",index=False)